<h1>contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#tl;dr" data-toc-modified-id="tl;dr-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>tl;dr</a></span></li><li><span><a href="#setup" data-toc-modified-id="setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#data" data-toc-modified-id="data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>data</a></span></li></ul></div>

# setup

In [1]:
import datetime
import gc
import json
import os
import sys
import pprint
import warnings
from io import BytesIO
import psutil

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import folium
import geopandas as gpd
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import netCDF4 as nc
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp 
import tqdm
from folium.plugins import HeatMap
from IPython.display import display
from scipy.interpolate import griddata
from selenium import webdriver
from skimage.transform import resize
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.preprocessing import MinMaxScaler

mpl.rcParams.update(mpl.rcParamsDefault)
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
print(os.uname())
print("Cores: ", os.cpu_count())
print("Memory: ", round(psutil.virtual_memory().total / 1024 / 1024 / 1024, 2), "GB")
print()
print(sys.version)
print(sys.version_info)
print()
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

posix.uname_result(sysname='Linux', nodename='u22', release='6.2.0-36-generic', version='#37~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  9 15:34:04 UTC 2', machine='x86_64')
Cores:  16
Memory:  15.54 GB

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)

2.0.0+cu117
11.7
8500
NVIDIA GeForce GTX 1660 Ti
True


**EXCERTO DO ARTIGO:**

_"O modelo foi desenvolvido em Ubuntu Linux 22.04 de 64 bits, com 16 GB de RAM e um processador AMD Ryzen 7 3700x 8-core processor x 16, de placa de vídeo GeForce GTX 1660 Ti. Todos os códigos foram interpretados em Python 3.10.12, em particular o modelo de predição de incêndios florestais foi desenvolvido com o auxílio da biblioteca PyTorch 2.0.0+cu117"._

In [119]:
def sizeof_fmt(num, suffix='B'):
    """
    Convert size of objects in memory
    """
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "{:>6.1f} {}{}".format(num, unit, suffix)
        num /= 1024.0
    return "{:.1f} {}{}".format(num, 'Yi', suffix)

In [120]:
sns.set_style("whitegrid")

plt.rcParams.update({
    'font.size'       : 9,
    'figure.figsize'  : (3.5, 2.8),
    'figure.dpi'      : 300,
    'savefig.dpi'     : 300,
    'text.usetex'     : True,
    'font.family'     : 'serif',
    'font.serif'      : ['Times New Roman'],
    'axes.labelsize'  : 9,
    'axes.titlesize'  : 9,
    'xtick.labelsize' : 8,
    'ytick.labelsize' : 8,
    'legend.fontsize' : 8,
    'lines.linewidth' : 1,
    'axes.linewidth'  : 1,
    'grid.linestyle'  : '--',
    'grid.linewidth'  : 0.5,
    'grid.alpha'      : 0.8,
})

sns.set_context(
    "paper", 
    rc={
        "font.size"      : 9,
        "axes.titlesize" : 9,
        "axes.labelsize" : 9, 
        'xtick.labelsize': 8,
        'ytick.labelsize': 8
    }
)

In [121]:
experiment_name = "wildfire_spread"
folder_path = f"../data/08_reporting/{experiment_name}"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

golden_ratio = (np.sqrt(5) - 1) / 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [122]:
BRAZIL_EXTENT = [-74, -34, -34, 6]
SAO_PAULO_EXTENT = [-53, -44, -25, -20]

# data

In [123]:
MAX_LAT = 32.99499999641088
MIN_LAT = -55.98500000358912
MAX_LON = -33.009801806103724
MIN_LON = -119.98980180610373

MAX_LAT = BRAZIL_EXTENT[3]
MIN_LAT = BRAZIL_EXTENT[2]
MAX_LON = BRAZIL_EXTENT[1]
MIN_LON = BRAZIL_EXTENT[0]

In [124]:
def convert_data_to_tensor_layer_feature(data, column_name:str):
    """
    Convert data to tensor layer feature
    """
    # LAT_LON_SHAPE = (3000, 3000)

    MAX_LAT = 32.99499999641088
    MIN_LAT = -55.98500000358912
    MAX_LON = -33.009801806103724
    MIN_LON = -119.98980180610373

    MAX_LAT = BRAZIL_EXTENT[3]
    MIN_LAT = BRAZIL_EXTENT[2]
    MAX_LON = BRAZIL_EXTENT[1]
    MIN_LON = BRAZIL_EXTENT[0]

    fig = plt.figure(figsize=(1, 1))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([MIN_LON, MAX_LON, MIN_LAT, MAX_LAT], crs=ccrs.PlateCarree(),)

    try:
        ax.imshow(
            np.flipud(data.variables[column_name][:].data[0]),
            origin="upper",
            extent=[MIN_LON, MAX_LON, MIN_LAT, MAX_LAT],
            transform=ccrs.PlateCarree(),
            cmap="gray",
        )
    except:
        ax.imshow(
            data.variables[column_name][:].data[0],
            origin="upper",
            extent=[MIN_LON, MAX_LON, MIN_LAT, MAX_LAT],
            transform=ccrs.PlateCarree(),
            cmap="gray",
        )
    
    fig.canvas.draw()
    
    data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    # gray_data = np.dot(data[...,:3], [0.2989, 0.5870, 0.1140])
    # gray_data_normalized = gray_data / 255.0
    sum_channel_data = np.sum(data, axis=2)

    plt.close(fig)

    return sum_channel_data

In [125]:
def convert_data_to_tensor_layer_target(longitude, latitude):
    """
    Convert data to tensor layer target
    """
    # LAT_LON_SHAPE = (2000, 2000)

    MAX_LAT = 32.99499999641088
    MIN_LAT = -55.98500000358912
    MAX_LON = -33.009801806103724
    MIN_LON = -119.98980180610373

    MAX_LAT = BRAZIL_EXTENT[3]
    MIN_LAT = BRAZIL_EXTENT[2]
    MAX_LON = BRAZIL_EXTENT[1]
    MIN_LON = BRAZIL_EXTENT[0]

    # fig = plt.figure(figsize=(3, 3))
    # ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    # ax.set_extent([MIN_LON, MAX_LON, MIN_LAT, MAX_LAT], crs=ccrs.PlateCarree(),)

    # ax.add_feature(cfeature.BORDERS, linewidth=0.05, edgecolor="white", zorder=2)
    # ax.add_feature(cfeature.COASTLINE, linewidth=0.05, edgecolor="white", zorder=2)
    
    _hist = np.histogram2d(
        longitude, latitude, bins=300, range=[[MIN_LON, MAX_LON], [MIN_LAT, MAX_LAT]]
    )
    _hist_plot = _hist[0] 
    _hist_plot = np.where(_hist_plot > 0, 1, 0)

    # ax.imshow(
    #     np.flipud(_hist_plot.T),
    #     origin="upper",
    #     extent=[MIN_LON, MAX_LON, MIN_LAT, MAX_LAT],
    #     transform=ccrs.PlateCarree(),
    # )
    # ax.set_axis_off()

    # data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    # data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    # # gray_data = np.dot(data[...,:3], [0.2989, 0.5870, 0.1140])
    # # gray_data_normalized = gray_data / 255.0
    # sum_channel_data = np.sum(data, axis=2)

    return np.flipud(_hist_plot.T)

**EXCERTO DO ARTIGO:**

_"Os dados considerados para o modelo foram obtidos no site do Projeto Queimadas do Instituto Nacional de Pesquisas Espaciais (INPE). Esse projeto possui medidas de sensoriamento remoto de incêndios florestais no Brasil desde 1998, com dados diários de focos de incêndio, que são obtidos por meio de satélites dez satélites polares e óticos operados em faixa termal-média de 4um. Esse conjunto produz aproximadamente 200 imagens por dia com periodicidades horárias e diárias, que posteriormente são reprocessadas e sumarizadas em periodicidade mensal._

_Após análise exploratória prévia, foram escolhidos os seguintes conjuntos de dados para o modelo:_

- _Focos de Incêndio: variável resposta composta por dados diários de focos de incêndio, baixados em formato CSV._
- _Precipitação: variável explicativa composta por dados diários de precipitação, baixados em formato netCDF, em unidades de medidas não explicitadas, de dimensões de latitude e longitude 901x850._
- _Temperatura: variável explicativa composta por dados diários de temperatura, baixados em formato netCDF, em unidades de medidas não explicitadas, de dimensões de latitude e longitude 361x345._
- _Umidade Relativa do Ar: variável explicativa composta por dados diários de umidade relativa do ar, baixados em formato netCDF, em unidades de medidas não explicitadas, de dimensões de latitude e longitude 361x345._
- _Número de Dias Sem Chuva: variável explicativa composta por número de dias sem chuva no período de 120 dias, baixados em formato netCDF, de dimensões de latitude e longitude 850x901._

_Embora haja histórico de mais de três meses armazenado na página de download dos conjuntos de dados do projeto, os conjuntos de focos de incêndio possuem disponibilidade menor, então a série temporal diária considerada foi restrita entre 19 de outubro de 2023 e 24 de novembro de 2023, conforme ilustrado na figura._

_Essas matrizes foram padronizadas para compor o conjunto de dados de entrada do modelo. Para cada um dos N dias de observação, as matrizes de variáveis explicativas listadas acima foram exibidas em um gráfico geográfico com projeção cilíndrica de Plate Carrée, com coordenadas restritas a longitudes e latitudes do território brasileiro. A imagem resultante desses gráficos foram "achatadas" através da soma dos valores dos canais RGB de cada pixel, resultando em matrizes de dimensões 300x300, que foram utilizadas como camadas de tensores de entrada do modelo. O tensor final possui a seguinte dimensionalidade: (N, 5, 300, 300). O pseudocodigo abaixo ilustra o processo de preparação dos dados para o modelo"_

```tex 
\begin{algorithm}
\caption{Preparação de Dados}
\begin{algorithmic}[1]
\Procedure{DataPrep}{$data, column\_name$}
    \State $MAX\_LAT \gets 32.99499999641088$
    \State $MIN\_LAT \gets -55.98500000358912$
    \State $MAX\_LON \gets -33.009801806103724$
    \State $MIN\_LON \gets -119.98980180610373$
    
    \State Cria figura com projeção PlateCarree
    \State Restringe coordenadas de acordo com constantes de longitude e latitude

    \State Exibe figura
    
    \State $data \gets$ extrai RGB da figura
    \State $sum\_channel\_data \gets$ soma canais de $data$

    \State \Return $sum\_channel\_data$
\EndProcedure
\end{algorithmic}
\end{algorithm}
```

_A construção da variável resposta foi feita de forma análoga, porém no lugar de criação de imagens geográficas, a abordagem se baseou na criação de histogramas bidimensionais de "bins" contidos nos mesmos intervalos de longitude e latitude das variáveis explicativas. Contudo isso faria com que o modelo a ser desenvolvido precisasse aproximar uma função distribuição de probabilidade multinomial. Para simplificar o problema, foi aplicada condição de binarização em que se um "bin" contiver pelo menos um foco de incêndio, o valor da variável resposta será 1, caso contrário será 0._

In [6]:
_file_path = f"../data/01_raw/inpe_queimadas/precipitacao/INPE_FireRiskModel_2.2_Precipitation_20230921.nc"
data = nc.Dataset(_file_path)
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    CDI: Climate Data Interface version 2.2.1 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    BeginDate: 2023-09-21
    BeginTime: 00:00:00.000Z
    EndDate: 2023-09-21
    EndTime: 23:59:59.999Z
    FileHeader: StartGranuleDateTime=2023-09-21T00:00:00.000Z;
StopGranuleDateTime=2023-09-21T23:59:59.999Z
    InputPointer: 3B-HHR-E.MS.MRG.3IMERG.20230921-S000000-E002959.0000.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S003000-E005959.0030.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S010000-E012959.0060.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S013000-E015959.0090.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S020000-E022959.0120.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S023000-E025959.0150.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S030000-E032959.0180.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S033000-E035959.0210.V06D.HDF5;3B-HHR-E.MS.MRG.3IMERG.20230921-S040000-E042959.0240.V06D.HDF5;3

In [8]:
_file_path = f"../data/01_raw/inpe_queimadas/temperatura/INPE_FireRiskModel_2.2_Temperature_20230921.nc"
data = nc.Dataset(_file_path)
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    CDI: Climate Data Interface version 2.2.1 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    institution: National Centers for Environmental Prediction
    history: Fri Sep 22 06:00:03 2023: cdo -s -setmissval,-999 -select,name=temp2m /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/RH2M.TEMP2M.GFS.ANL.2023092118.nc /home/queimadas/INPE_FireRiskModel/data/output/2.2/Temperature-2_2/2023/09/INPE_FireRiskModel_2.2_Temperature_20230921.nc
Fri Sep 22 06:00:03 2023: cdo -s -O merge /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/tmp003.var.temp2m.nc /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/tmp003.var.rh2m.nc /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/RH2M.TEMP2M.GFS.ANL.2023092118.nc
Fri Sep 22 06:00:03 2023: cdo -s splitvar /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/tmp002.GFS.ANL.2023092118.nc /home/queim

In [2]:
_file_path = f"../data/01_raw/inpe_queimadas/umidade_relativa/INPE_FireRiskModel_2.2_RelativeHumidity_20230921.nc"
data = nc.Dataset(_file_path)
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    CDI: Climate Data Interface version 2.2.1 (https://mpimet.mpg.de/cdi)
    Conventions: CF-1.6
    institution: National Centers for Environmental Prediction
    history: Fri Sep 22 06:00:03 2023: cdo -s -setmissval,-999 -select,name=rh2m /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/RH2M.TEMP2M.GFS.ANL.2023092118.nc /home/queimadas/INPE_FireRiskModel/data/output/2.2/RelativeHumidity-2_2/2023/09/INPE_FireRiskModel_2.2_RelativeHumidity_20230921.nc
Fri Sep 22 06:00:03 2023: cdo -s -O merge /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/tmp003.var.temp2m.nc /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/tmp003.var.rh2m.nc /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/RH2M.TEMP2M.GFS.ANL.2023092118.nc
Fri Sep 22 06:00:03 2023: cdo -s splitvar /home/queimadas/INPE_FireRiskModel/scr/Temp_UR_download/tmp/tmp002.GFS.ANL.2023092118.nc /ho

In [3]:
_file_path = f"../data/01_raw/inpe_queimadas/numero_dias_sem_chuva/INPE_FireRiskModel_2.2_NDWR_20230921.nc"
data = nc.Dataset(_file_path)
print(data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    creation_date: Fri 22 Sep 2023 09:06:02 AM UTC
    source: Codigo feito no software NCL 6.6.2
    link: http://www.inpe.br/queimadas
    codigo: Guilherme Martins - queimadas@inpe.br
    Conventions: None
    title: Numero de dias sem chuva (NDSC) no período de 120 dias
    dimensions(sizes): time(1), lat(901), lon(850)
    variables(dimensions): float32 ndsc120(time, lat, lon), float64 time(time), float32 lat(lat), float32 lon(lon)
    groups: 


In [126]:
# LAT_LON_SHAPE = (2000, 2000)
# # TENSOR = np.empty((0, 6, LAT_LON_SHAPE[0], LAT_LON_SHAPE[1]))
# TENSOR = np.empty((0, 5, LAT_LON_SHAPE[0], LAT_LON_SHAPE[1]))
TENSOR_TIMESTAMPS = list()

# for _dt in tqdm.tqdm(pd.date_range("2023-08-01", "2023-12-01")):
for _dt in tqdm.tqdm(pd.date_range("2023-08-01", "2023-12-01")):
    try:
        print()
        # fire occurence
        _dataset = pd.read_csv(f"../data/01_raw/inpe_queimadas/focos/focos_diario_br_{_dt.strftime('%Y%m%d')}.csv")
        # if _dataset.query("risco_fogo > 0").shape[0] > 0:
        if _dataset.shape[0] > 0:
            _tensor = list()

            # _dataset = _dataset.query("risco_fogo >= 0").copy()
            data = convert_data_to_tensor_layer_target(_dataset["lon"], _dataset["lat"])
            print("fire occurence", data.shape)
            _tensor.append(data)

            # loading num_dias_sem_chuva
            _file_path = f"../data/01_raw/inpe_queimadas/numero_dias_sem_chuva/INPE_FireRiskModel_2.2_NDWR_{_dt.strftime('%Y%m%d')}.nc"
            _dataset = nc.Dataset(_file_path)
            data = convert_data_to_tensor_layer_feature(_dataset, "ndsc120")
            print("num_dias_sem_chuva", data.shape)
            _tensor.append(data)

            # loading precipitacao
            _file_path = f"../data/01_raw/inpe_queimadas/precipitacao/INPE_FireRiskModel_2.2_Precipitation_{_dt.strftime('%Y%m%d')}.nc"
            _dataset = nc.Dataset(_file_path)
            data = convert_data_to_tensor_layer_feature(_dataset, "prec")
            print("precipitacao", data.shape)
            _tensor.append(data)

            # loading temperatura
            _file_path = f"../data/01_raw/inpe_queimadas/temperatura/INPE_FireRiskModel_2.2_Temperature_{_dt.strftime('%Y%m%d')}.nc"
            _dataset = nc.Dataset(_file_path)
            data = convert_data_to_tensor_layer_feature(_dataset, "temp2m")
            print("temperatura", data.shape)
            _tensor.append(data)

            # loading umidade
            _file_path = f"../data/01_raw/inpe_queimadas/umidade_relativa/INPE_FireRiskModel_2.2_RelativeHumidity_{_dt.strftime('%Y%m%d')}.nc"
            _dataset = nc.Dataset(_file_path)
            data = convert_data_to_tensor_layer_feature(_dataset, "rh2m")
            print("umidade", data.shape)
            _tensor.append(data)

            # # loading risco de fogo
            # _file_path = f"../data/01_raw/inpe_queimadas/risco_fogo/INPE_FireRiskModel_2.2_FireRisk_{_dt.strftime('%Y%m%d')}.nc"
            # _dataset = nc.Dataset(_file_path)
            # data = convert_data_to_tensor_layer_feature(_dataset, "rf")
            # print("risco de fogo", data.shape)
            # _tensor.append(data)

            # TENSOR = np.append(TENSOR, np.array([_tensor]), axis=0)
            # TENSOR_TIMESTAMPS.append(_dt.strftime("%Y-%m-%d"))
            print(np.array(_tensor).shape)

            # save as compressed npz
            np.savez_compressed(
                f"../data/02_intermediate/wildfire_spread/{_dt.strftime('%Y%m%d')}.npz",
                tensor=np.array(_tensor),
                timestamps=_dt.strftime("%Y-%m-%d"),
            )
            
            # clean temporary variables and memory
            del _tensor
            del data
            del _dataset
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
    except Exception as e:
        # print(e)
        continue


  0%|          | 0/123 [00:00<?, ?it/s]

















































































fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)


 65%|██████▌   | 80/123 [00:00<00:00, 164.40it/s]


fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_c

 79%|███████▉  | 97/123 [00:05<00:02, 12.48it/s] 

...


fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)

fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)


 93%|█████████▎| 115/123 [00:12<00:01,  5.45it/s]


fire occurence (300, 300)
num_dias_sem_chuva (300, 300)
precipitacao (300, 300)
temperatura (300, 300)
umidade (300, 300)
(5, 300, 300)


100%|██████████| 123/123 [00:12<00:00,  9.76it/s]


fire occurence (300, 300)








In [127]:
TENSOR = torch.empty((0, 5, 300, 300), dtype=torch.float16)

for _dt in pd.date_range("2023-08-01", "2023-12-01", freq="D"):
    try:
        _data = np.load(
            f"../data/02_intermediate/wildfire_spread/{_dt.strftime('%Y%m%d')}.npz",
        )
        print(_dt.strftime("%Y-%m-%d"))
        _timestamp = _data["timestamps"]
        _tensor = _data["tensor"].astype(np.float16)
        _tensor = torch.from_numpy(_tensor)
        _tensor = _tensor.unsqueeze(0)
        TENSOR = torch.cat((TENSOR, _tensor), dim=0)
        del _tensor
        del _data
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    except Exception as e:
        # print(e)
        continue

2023-10-19
2023-10-20
2023-10-21
2023-10-22
2023-10-23
2023-10-24
2023-10-25
2023-10-26
2023-10-27
2023-10-28
2023-10-29
2023-10-30
2023-10-31
2023-11-01
2023-11-02
2023-11-03
2023-11-04
2023-11-05
2023-11-06
2023-11-07
2023-11-08
2023-11-09
2023-11-10
2023-11-11
2023-11-12
2023-11-13
2023-11-14
2023-11-15
2023-11-16
2023-11-17
2023-11-18
2023-11-19
2023-11-20
2023-11-21
2023-11-22
2023-11-23
2023-11-24


In [128]:
torch.save(TENSOR, f"../data/02_intermediate/wildfire_spread/wildfire_spread_tensor.pt")

<br/><br/><br/><br/><br/>

In [16]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                      _dataset:    3.6 MiB
                          data:  703.2 KiB
                          _iii:    3.4 KiB
                          _i13:    3.4 KiB
                           _i8:    1.5 KiB
                          _i11:    1.5 KiB
                          _i12:    1.1 KiB
                       HeatMap:    1.0 KiB
                 ParameterGrid:    1.0 KiB
                  MinMaxScaler:    1.0 KiB
